Q2.1 Classic Machine Learning Methods

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from sklearn.metrics import roc_auc_score, average_precision_score
from tsfresh.feature_extraction import extract_features, MinimalFCParameters
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.feature_extraction import MinimalFCParameters
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# === PART 1 OF Q2.1 ===
# Load preprocessed datasets
df_a = pd.read_parquet('ml_ready_data/set-a-scaled.parquet')
df_b = pd.read_parquet('ml_ready_data/set-b-scaled.parquet')
df_c = pd.read_parquet('ml_ready_data/set-c-scaled.parquet')  # Test set

df_train = df_a
df_validate = df_b
df_test = df_c  # Test set is set-c

# Define static and dynamic variables
static_vars = ['Age', 'Gender', 'Height', 'Weight']
time_series_vars = [col for col in df_train.columns if col not in static_vars + ['Hour', 'PatientID', 'RecordID', 'In_hospital_death']]

# Feature extraction function
def extract_features(df):
    features = df.groupby('PatientID')[time_series_vars].agg(['mean', 'max', 'last'])
    features.columns = ['_'.join(col) for col in features.columns]
    
    # Add static variables
    static_data = df.groupby('PatientID')[static_vars].last()
    features = features.merge(static_data, left_index=True, right_index=True)
    
    # Add labels
    labels = df.groupby('PatientID')['In_hospital_death'].last()
    features['In_hospital_death'] = labels
    return features.reset_index()

# Extract features
df_train_features = extract_features(df_train)
df_test_features = extract_features(df_test)

# Prepare data for ML
X_train = df_train_features.drop(columns=['PatientID', 'In_hospital_death'])
y_train = df_train_features['In_hospital_death']
X_test = df_test_features.drop(columns=['PatientID', 'In_hospital_death'])
y_test = df_test_features['In_hospital_death']

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train classifiers
logreg = LogisticRegression()
rf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_model = SVC(kernel='linear', probability=True, random_state=42)

logreg.fit(X_train, y_train)
rf.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# Evaluate models
def evaluate_model(model, X_test, y_test, name):
    y_pred = model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred)
    auprc = average_precision_score(y_test, y_pred)
    print(f"{name} - AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")

# Report performance
evaluate_model(logreg, X_test, y_test, 'Logistic Regression')
evaluate_model(rf, X_test, y_test, 'Random Forest')
evaluate_model(svm_model, X_test, y_test, 'SVM')


Logistic Regression - AUROC: 0.8468, AUPRC: 0.5152
Random Forest - AUROC: 0.8346, AUPRC: 0.5078
SVM - AUROC: 0.8309, AUPRC: 0.4873


In [3]:
#print(ts_features.shape)
df_long = df_train[['PatientID', 'Hour'] + time_series_vars]


In [4]:
from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, average_precision_score

# === PART 2 OF Q2.1 ===

# Extract features with tsfresh
df_train = df_a
df_validate = df_b
df_test = df_c  # Test set is set-c

# Define static and dynamic variables
static_vars = ['Age', 'Gender', 'Height', 'Weight']
time_series_vars = [col for col in df_train.columns if col not in static_vars + ['Hour', 'PatientID', 'RecordID', 'In_hospital_death']]

# Extract features with tsfresh
def extract_tsfresh_features(df):
    # Extract time-series features using tsfresh
    ts_features = extract_features(
        df[time_series_vars + static_vars + ['PatientID', 'Hour']],  # Include necessary columns
        column_id='PatientID',  # ID column for each patient
        column_sort='Hour',  # Time-related sorting column
        default_fc_parameters=MinimalFCParameters(),  # Set of features to extract
        n_jobs=1  # Adjust as needed for parallel processing
    )
    
    # Add static variables (like Age, Gender, Height, Weight) to the features
    static_data = df.groupby('PatientID')[static_vars].last()  # Get last static data for each patient
    ts_features = ts_features.merge(static_data, left_index=True, right_index=True)
    
    # Add the target variable 'In_hospital_death'
    labels = df.groupby('PatientID')['In_hospital_death'].last()  # Get the last In_hospital_death value for each patient
    ts_features['In_hospital_death'] = labels
    
    return ts_features.reset_index()

# Extract features for training and testing sets using tsfresh
df_train_features = extract_tsfresh_features(df_a)
df_test_features = extract_tsfresh_features(df_c)

# Prepare data for ML
X_train = df_train_features.drop(columns=['In_hospital_death'])
y_train = df_train.groupby('PatientID')['In_hospital_death'].last()
X_test = df_test_features.drop(columns=['In_hospital_death'])
y_test = df_test.groupby('PatientID')['In_hospital_death'].last()

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train classifiers
logreg = LogisticRegression()
rf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_model = SVC(kernel='linear', probability=True, random_state=42)

logreg.fit(X_train, y_train)
rf.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# Evaluate models
def evaluate_model(model, X_test, y_test, name):
    y_pred = model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred)
    auprc = average_precision_score(y_test, y_pred)
    print(f"{name} - AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")

# Report performance
evaluate_model(logreg, X_test, y_test, 'Logistic Regression')
evaluate_model(rf, X_test, y_test, 'Random Forest')
evaluate_model(svm_model, X_test, y_test, 'SVM')


Feature Extraction: 100%|██████████| 164000/164000 [00:40<00:00, 4021.38it/s]
/cluster/courses/ml4h/jupyter/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression - AUROC: 0.8109, AUPRC: 0.4315
Random Forest - AUROC: 0.8162, AUPRC: 0.4654
SVM - AUROC: 0.7862, AUPRC: 0.3974


Q2.2 Recurrent Neural Networks

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

# === Implementing LSTM ===

# === Prepare Data for LSTM ===
def prepare_lstm_data(df, time_series_vars, static_vars):
    time_series_data = df.groupby('PatientID')[time_series_vars].apply(lambda x: x.values)
    static_data = df.groupby('PatientID')[static_vars].last()
    labels = df.groupby('PatientID')['In_hospital_death'].last()

    # Pad sequences to ensure uniform length
    max_timesteps = max(time_series_data.apply(len))
    time_series_data = np.array([np.pad(x, ((0, max_timesteps - len(x)), (0, 0)), mode='constant') for x in time_series_data])

    return time_series_data, static_data.values, labels.values

# === Prepare Training and Test Data ===
X_train_ts, X_train_static, y_train = prepare_lstm_data(df_a, time_series_vars, static_vars)
X_test_ts, X_test_static, y_test = prepare_lstm_data(df_c, time_series_vars, static_vars)

# === Standardize Features ===
scaler_static = StandardScaler()
X_train_static = scaler_static.fit_transform(X_train_static)
X_test_static = scaler_static.transform(X_test_static)

scaler_ts = StandardScaler()
X_train_ts = np.array([scaler_ts.fit_transform(x) for x in X_train_ts])
X_test_ts = np.array([scaler_ts.transform(x) for x in X_test_ts])

# === Convert to PyTorch Tensors ===
X_train_ts_tensor = torch.tensor(X_train_ts, dtype=torch.float32)
X_test_ts_tensor = torch.tensor(X_test_ts, dtype=torch.float32)
X_train_static_tensor = torch.tensor(X_train_static, dtype=torch.float32)
X_test_static_tensor = torch.tensor(X_test_static, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# === Define the LSTM Model ===
class LSTMModel(nn.Module):
    def __init__(self, input_shape_ts, input_shape_static):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_shape_ts[1], hidden_size=64, batch_first=True, dropout=0.3)
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=32, batch_first=True, dropout=0.3)
        self.fc1 = nn.Linear(32 + input_shape_static, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x_ts, x_static):
        lstm_out, (hn, cn) = self.lstm1(x_ts)
        lstm_out, (hn, cn) = self.lstm2(lstm_out)
        last_lstm_output = lstm_out[:, -1, :]  # Use the last time-step output

        # Concatenate static data with LSTM output
        combined_input = torch.cat((last_lstm_output, x_static), dim=1)
        x = torch.relu(self.fc1(combined_input))
        x = self.fc2(x)
        return self.sigmoid(x)

# === Initialize and Train the Model ===
input_shape_ts = (X_train_ts.shape[1], X_train_ts.shape[2])  # (timesteps, features)
input_shape_static = X_train_static.shape[1]  # Number of static features

model = LSTMModel(input_shape_ts, input_shape_static)
criterion = nn.BCELoss()  # Binary Cross-Entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === Training Loop ===
def train_model(model, X_train_ts, X_train_static, y_train, epochs=20, batch_size=32):
    model.train()
    for epoch in range(epochs):
        # Create batches
        for i in range(0, len(X_train_ts), batch_size):
            batch_ts = X_train_ts[i:i+batch_size]
            batch_static = X_train_static[i:i+batch_size]
            batch_labels = y_train[i:i+batch_size]
            
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(batch_ts, batch_static)
            loss = criterion(outputs.squeeze(), batch_labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
        
        # Print training progress
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# === Train the Model ===
train_model(model, X_train_ts_tensor, X_train_static_tensor, y_train_tensor)

# === Evaluate the Model ===
model.eval()
with torch.no_grad():
    y_pred = model(X_test_ts_tensor, X_test_static_tensor).squeeze()
    y_pred = y_pred.numpy()
    
    auroc = roc_auc_score(y_test, y_pred)
    auprc = average_precision_score(y_test, y_pred)

print(f"LSTM Model - Test Set C Performance: AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")


/cluster/courses/ml4h/jupyter/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Epoch [1/20], Loss: 0.4644
Epoch [2/20], Loss: 0.3617
Epoch [3/20], Loss: 0.3375
Epoch [4/20], Loss: 0.3165
Epoch [5/20], Loss: 0.3092
Epoch [6/20], Loss: 0.2887
Epoch [7/20], Loss: 0.2658
Epoch [8/20], Loss: 0.2290
Epoch [9/20], Loss: 0.1945
Epoch [10/20], Loss: 0.1713
Epoch [11/20], Loss: 0.1578
Epoch [12/20], Loss: 0.2450
Epoch [13/20], Loss: 0.1078
Epoch [14/20], Loss: 0.0806
Epoch [15/20], Loss: 0.0309
Epoch [16/20], Loss: 0.1565
Epoch [17/20], Loss: 0.0549
Epoch [18/20], Loss: 0.0117
Epoch [19/20], Loss: 0.0072
Epoch [20/20], Loss: 0.0092
LSTM Model - Test Set C Performance: AUROC: 0.6841, AUPRC: 0.2522


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score, average_precision_score
from torch.utils.data import DataLoader, TensorDataset

# === Define the Bidirectional LSTM Model in PyTorch ===
class BidirectionalLSTM(nn.Module):
    def __init__(self, input_dim_ts, hidden_dim=64, dropout_rate=0.3):
        super(BidirectionalLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_dim_ts, hidden_dim, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_dim*2, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_dim*2, 16)
        self.fc2 = nn.Linear(16, 1)  # Binary classification
    
    def forward(self, x):
        # Pass through first bidirectional LSTM
        x, _ = self.lstm1(x)  
        x = self.dropout(x)
        
        # Pass through second bidirectional LSTM
        x, _ = self.lstm2(x)
        x = self.dropout(x)
        
        # Take the output from the last timestep
        x = x[:, -1, :]
        
        # Fully connected layers
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        
        return torch.sigmoid(x)  # Sigmoid for binary classification

# === Data Preparation for PyTorch ===
def prepare_lstm_data_for_pytorch(df, time_series_vars, static_vars):
    time_series_data, static_data, labels = prepare_lstm_data(df, time_series_vars, static_vars)
    
    # Convert to PyTorch tensors
    time_series_data = torch.tensor(time_series_data, dtype=torch.float32)
    static_data = torch.tensor(static_data, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)
    
    return time_series_data, static_data, labels

# Prepare training and test data
X_train_ts, X_train_static, y_train = prepare_lstm_data_for_pytorch(df_a, time_series_vars, static_vars)
X_test_ts, X_test_static, y_test = prepare_lstm_data_for_pytorch(df_c, time_series_vars, static_vars)

# Create datasets and dataloaders for training and testing
train_dataset = TensorDataset(X_train_ts, X_train_static, y_train)
test_dataset = TensorDataset(X_test_ts, X_test_static, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# === Define the Training Function ===
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        epoch_loss = 0
        for batch in train_loader:
            time_series_data, _, labels = batch
            
            optimizer.zero_grad()  # Clear previous gradients
            outputs = model(time_series_data)  # Forward pass
            
            loss = criterion(outputs.squeeze(), labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            epoch_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

# === Train the Model ===
# Initialize the model, loss function, and optimizer
input_dim_ts = X_train_ts.shape[2]  # Number of time series features
model = BidirectionalLSTM(input_dim_ts)

criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, criterion, optimizer, epochs=20)

# === Evaluate the Model ===
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for batch in test_loader:
            time_series_data, _, labels = batch
            outputs = model(time_series_data)
            
            y_true.extend(labels.numpy())
            y_pred.extend(outputs.squeeze().numpy())
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    auroc = roc_auc_score(y_true, y_pred)
    auprc = average_precision_score(y_true, y_pred)
    
    return auroc, auprc

# Evaluate the model on the test set
auroc, auprc = evaluate_model(model, test_loader)
print(f"Bidirectional LSTM Model - Test Set C Performance: AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")


Epoch 1/20, Loss: 0.4288
Epoch 2/20, Loss: 0.3454
Epoch 3/20, Loss: 0.3211
Epoch 4/20, Loss: 0.2941
Epoch 5/20, Loss: 0.2726
Epoch 6/20, Loss: 0.2497
Epoch 7/20, Loss: 0.2283
Epoch 8/20, Loss: 0.2054
Epoch 9/20, Loss: 0.1916
Epoch 10/20, Loss: 0.1792
Epoch 11/20, Loss: 0.1666
Epoch 12/20, Loss: 0.1513
Epoch 13/20, Loss: 0.1379
Epoch 14/20, Loss: 0.1264
Epoch 15/20, Loss: 0.1136
Epoch 16/20, Loss: 0.1125
Epoch 17/20, Loss: 0.1070
Epoch 18/20, Loss: 0.0844
Epoch 19/20, Loss: 0.0955
Epoch 20/20, Loss: 0.0886
Bidirectional LSTM Model - Test Set C Performance: AUROC: 0.7373, AUPRC: 0.2933


Q2.3a Transformers

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np
import pandas as pd

# Define Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, time_series_input_size, static_input_size, num_heads=4, hidden_dim=64, projected_dim=40):
        super(TransformerModel, self).__init__()
        
        # Project time-series features to an embedding size divisible by num_heads
        self.input_projection = nn.Linear(time_series_input_size, projected_dim)
        
        # Multi-Head Attention Layer
        self.attention_layer = nn.MultiheadAttention(embed_dim=projected_dim, num_heads=num_heads, batch_first=True)
        
        # Fully connected layers after attention
        self.fc1 = nn.Linear(projected_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        
        # Process static features
        self.static_fc = nn.Linear(static_input_size, hidden_dim)
        
        # Output layer
        self.output_fc = nn.Linear(hidden_dim, 1)
        
        # Dropout
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, time_series_input, static_input):
        # Project time-series features
        time_series_input = self.input_projection(time_series_input)
        
        # Apply attention
        attention_output, _ = self.attention_layer(time_series_input, time_series_input, time_series_input)
        
        # Pooling
        x = attention_output.mean(dim=1)  # Global average pooling
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        
        # Process static features
        static_out = self.static_fc(static_input)
        
        # Combine time-series and static features
        combined = x + static_out  # Element-wise addition
        combined = self.dropout(combined)
        
        # Final output
        output = self.output_fc(combined)
        return torch.sigmoid(output)

# Load datasets
df_a = pd.read_parquet('ml_ready_data/set-a-scaled.parquet')
df_b = pd.read_parquet('ml_ready_data/set-b-scaled.parquet')
df_c = pd.read_parquet('ml_ready_data/set-c-scaled.parquet')

df_train = df_a
df_validate = df_b
df_test = df_c

# Define static and time-series variables
static_vars = ['Age', 'Gender', 'Height', 'Weight']
time_series_vars = [col for col in df_train.columns if col not in static_vars + ['Hour', 'PatientID', 'RecordID', 'In_hospital_death']]

def prepare_data(df):
    grouped = df.groupby('PatientID')
    
    # Time-series tensor: (num_patients, seq_len, num_features)
    time_series_data = np.array([group[time_series_vars].values for _, group in grouped])
    static_data = grouped[static_vars].last().values  # Static features (num_patients, num_static_features)
    labels = grouped['In_hospital_death'].last().values  # Labels
    
    return time_series_data, static_data, labels

# Prepare training & test sets
X_train_ts, X_train_static, y_train = prepare_data(df_train)
X_test_ts, X_test_static, y_test = prepare_data(df_test)

# Standardization
scaler_ts = StandardScaler()
scaler_static = StandardScaler()

X_train_ts = np.array([scaler_ts.fit_transform(seq) for seq in X_train_ts])
X_test_ts = np.array([scaler_ts.transform(seq) for seq in X_test_ts])

X_train_static = scaler_static.fit_transform(X_train_static)
X_test_static = scaler_static.transform(X_test_static)

# Convert to PyTorch tensors
X_train_ts = torch.tensor(X_train_ts, dtype=torch.float32)
X_train_static = torch.tensor(X_train_static, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_test_ts = torch.tensor(X_test_ts, dtype=torch.float32)
X_test_static = torch.tensor(X_test_static, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train_ts, X_train_static, y_train)
test_dataset = TensorDataset(X_test_ts, X_test_static, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Model Initialization
model = TransformerModel(time_series_input_size=len(time_series_vars), static_input_size=len(static_vars))
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_ts, batch_static, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_ts, batch_static)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
y_preds = []
y_true = []
with torch.no_grad():
    for batch_ts, batch_static, batch_y in test_loader:
        output = model(batch_ts, batch_static).cpu().numpy()
        y_preds.extend(output)
        y_true.extend(batch_y.cpu().numpy())

y_preds = np.array(y_preds).flatten()
y_true = np.array(y_true).flatten()

auroc = roc_auc_score(y_true, y_preds)
auprc = average_precision_score(y_true, y_preds)
print(f"Transformer Model - Test Set Performance: AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")


Epoch 1/20, Loss: 0.4424
Epoch 2/20, Loss: 0.3769
Epoch 3/20, Loss: 0.3659
Epoch 4/20, Loss: 0.3565
Epoch 5/20, Loss: 0.3453
Epoch 6/20, Loss: 0.3391
Epoch 7/20, Loss: 0.3323
Epoch 8/20, Loss: 0.3266
Epoch 9/20, Loss: 0.3173
Epoch 10/20, Loss: 0.3138
Epoch 11/20, Loss: 0.3027
Epoch 12/20, Loss: 0.2955
Epoch 13/20, Loss: 0.2879
Epoch 14/20, Loss: 0.2743
Epoch 15/20, Loss: 0.2648
Epoch 16/20, Loss: 0.2539
Epoch 17/20, Loss: 0.2455
Epoch 18/20, Loss: 0.2393
Epoch 19/20, Loss: 0.2235
Epoch 20/20, Loss: 0.2264
Transformer Model - Test Set Performance: AUROC: 0.3562, AUPRC: 0.1118


Q2.3b Tokenizing Time-Series Data and Transformers 

In [19]:
from sklearn.preprocessing import OneHotEncoder, RobustScaler
import numpy as np
import pandas as pd

# Assuming 'df_a', 'df_b', 'df_c' are already loaded
df_train = df_a
df_validate = df_b
df_test = df_c

# Define static and time-series variables
static_vars = ['Age', 'Gender', 'Height', 'Weight']
time_series_vars = [col for col in df_train.columns if col not in static_vars + ['Hour', 'PatientID', 'RecordID', 'In_hospital_death']]

# Normalize time for each dataset
df_train['t'] = df_train['Hour'] / df_train['Hour'].max()
df_validate['t'] = df_validate['Hour'] / df_train['Hour'].max()
df_test['t'] = df_test['Hour'] / df_train['Hour'].max()

# One-hot encode variable types (z)
enc = OneHotEncoder(sparse_output=False)
z_encoded = enc.fit_transform(np.array(time_series_vars).reshape(-1, 1))
var_dict = {var: z_encoded[i] for i, var in enumerate(time_series_vars)}

def process_patient_data(patient_df, scalers, max_seq_len=500):
    # Normalize time (t)
    t = patient_df['t'].values
    
    # Scale values (v) for all time_series_vars at once
    v_scaled = np.column_stack([
        scalers[var].transform(patient_df[[var]].fillna(0))[:, 0] for var in time_series_vars
    ])

    # Encode variable types (z) for all time_series_vars
    z_encoded = np.array([var_dict[var] for var in time_series_vars])

    # Repeat t for each variable and flatten
    t_repeated = np.repeat(t, len(time_series_vars))

    # Tile z_encoded for each time step
    z_tiled = np.tile(z_encoded, (len(t), 1))

    # Flatten v_scaled
    v_flattened = v_scaled.flatten()

    # Combine into triplets (t, z, v)
    triplets = [(t_repeated[i], z_tiled[i], v_flattened[i]) for i in range(len(t_repeated))]

    return triplets

# Fit scalers on training set
scalers = {var: StandardScaler().fit(df_train[[var]].dropna()) for var in time_series_vars}

# Process datasets
patient_groups_train = df_train.groupby('PatientID')
patient_groups_validate = df_validate.groupby('PatientID')
patient_groups_test = df_test.groupby('PatientID')

all_patients_train = [process_patient_data(group, scalers) for _, group in patient_groups_train]
all_patients_validate = [process_patient_data(group, scalers) for _, group in patient_groups_validate]
all_patients_test = [process_patient_data(group, scalers) for _, group in patient_groups_test]

# Convert to PyTorch tensors
X_train_ts = [torch.tensor([x[0] for x in seq], dtype=torch.float32).unsqueeze(1) for seq in all_patients_train]
X_train_z = [torch.tensor([x[1] for x in seq], dtype=torch.float32) for seq in all_patients_train]
X_train_v = [torch.tensor([x[2] for x in seq], dtype=torch.float32).unsqueeze(1) for seq in all_patients_train]

X_validate_ts = [torch.tensor([x[0] for x in seq], dtype=torch.float32).unsqueeze(1) for seq in all_patients_validate]
X_validate_z = [torch.tensor([x[1] for x in seq], dtype=torch.float32) for seq in all_patients_validate]
X_validate_v = [torch.tensor([x[2] for x in seq], dtype=torch.float32).unsqueeze(1) for seq in all_patients_validate]

X_test_ts = [torch.tensor([x[0] for x in seq], dtype=torch.float32).unsqueeze(1) for seq in all_patients_test]
X_test_z = [torch.tensor([x[1] for x in seq], dtype=torch.float32) for seq in all_patients_test]
X_test_v = [torch.tensor([x[2] for x in seq], dtype=torch.float32).unsqueeze(1) for seq in all_patients_test]

# Dataset preparation (combining t, z, v into a single tensor)
train_data = [(torch.cat((t, z, v), dim=1), torch.tensor(0.0)) for t, z, v in zip(X_train_ts, X_train_z, X_train_v)]
validate_data = [(torch.cat((t, z, v), dim=1), torch.tensor(0.0)) for t, z, v in zip(X_validate_ts, X_validate_z, X_validate_v)]
test_data = [(torch.cat((t, z, v), dim=1), torch.tensor(0.0)) for t, z, v in zip(X_test_ts, X_test_z, X_test_v)]

# Convert to DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
validate_loader = DataLoader(validate_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

class TransformerModel(nn.Module):
    def __init__(self, time_series_input_size, num_heads=4, hidden_dim=64, projected_dim=40):
        super(TransformerModel, self).__init__()
        
        # Project time-series features to an embedding size divisible by num_heads
        self.input_projection = nn.Linear(time_series_input_size, projected_dim)
        
        # Multi-Head Attention Layer
        self.attention_layer = nn.MultiheadAttention(embed_dim=projected_dim, num_heads=num_heads, batch_first=True)
        
        # Fully connected layers after attention
        self.fc1 = nn.Linear(projected_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        
        # Output layer
        self.output_fc = nn.Linear(hidden_dim, 1)
        
        # Dropout
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, time_series_input):
        # Apply attention
        attention_output, _ = self.attention_layer(time_series_input, time_series_input, time_series_input)
        
        # Pooling
        x = attention_output.mean(dim=1)  # Global average pooling
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        
        # Final output
        output = self.output_fc(x)
        return torch.sigmoid(output)
# Training Loop
epochs = 20
model = TransformerModel(time_series_input_size=1 + 41 + 1)  # t, z, v
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
y_preds = []
y_true = []
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        output = model(batch_x).cpu().numpy()
        y_preds.extend(output)
        y_true.extend(batch_y.cpu().numpy())

y_preds = np.array(y_preds).flatten()
y_true = np.array(y_true).flatten()

auroc = roc_auc_score(y_true, y_preds)
auprc = average_precision_score(y_true, y_preds)
print(f"Transformer Model - Test Set Performance: AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")



1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 